In [ ]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import pandas as pd
from datetime import datetime


In [ ]:
# Load age and emotion models
age_model = load_model('age_model.h5')
emotion_model = load_model('emotion_model.h5')


In [ ]:
# Capture video
cap = cv2.VideoCapture(0)

data = []

In [ ]:
while True:
    ret, frame = cap.read()
    face_locations = face_recognition.face_locations(frame)

    for (top, right, bottom, left) in face_locations:
        face = frame[top:bottom, left:right]
        face_resized = cv2.resize(face, (64, 64))
        face_normalized = face_resized / 255.0
        face_input = np.expand_dims(face_normalized, axis=0)

        age = age_model.predict(face_input)
        emotion = emotion_model.predict(face_input)

        if age < 13 or age > 60:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.putText(frame, 'Not Allowed', (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        else:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, f'Age: {age}, Emotion: {emotion}', (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        data.append((age, emotion, datetime.now()))

    cv2.imshow('Movie Theatre', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
# Save data to CSV
df = pd.DataFrame(data, columns=['Age', 'Emotion', 'Time'])
df.to_csv('movie_theatre.csv', index=False)

cap.release()
cv2.destroyAllWindows()